In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
%%time

df=pd.read_csv("en.openfoodfacts.org.products.csv",
              sep="\t", encoding="utf-8", low_memory=False)

In [4]:
%%time

df

CPU times: total: 0 ns
Wall time: 0 ns


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,product_name,abbreviated_product_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,nutrinet-sante,jeunes pousses,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1656948610,2022-07-04T15:30:10Z,1656948613,2022-07-04T15:30:13Z,kiliweb,Andrè,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1656488106,2022-06-29T07:35:06Z,stephane,L.casei,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1654250311,2022-06-03T09:58:31Z,1654270474,2022-06-03T15:34:34Z,quentinbrd,Skyr,NaN,...,NaN,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,kiliweb,Vitória crackers,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830750,3292590606497,http://world-en.openfoodfacts.org/product/3292...,kiliweb,1509909912,2017-11-05T19:25:12Z,1681141329,2023-04-10T15:42:09Z,moon-rabbit,Instant feerique,NaN,...,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830751,35033264,http://world-en.openfoodfacts.org/product/3503...,kiliweb,1583067829,2020-03-01T13:03:49Z,1681320470,2023-04-12T17:27:50Z,xdesnoyers,DooWap,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830752,39240958,http://world-en.openfoodfacts.org/product/3924...,kiliweb,1522995251,2018-04-06T06:14:11Z,1681320420,2023-04-12T17:27:00Z,xdesnoyers,Doowap,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830753,72355671,http://world-en.openfoodfacts.org/product/7235...,kiliweb,1558252418,2019-05-19T07:53:38Z,1680877496,2023-04-07T14:24:56Z,itsjustruby,Almond Butter,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Comparaison nombre null entre plusieurs colonnes semblables

### Packaging

In [5]:
df['packaging'].isna().sum()

2457066

In [25]:
df['packaging_tags'].isna().sum()

2457073

In [6]:
df['packaging_en'].isna().sum()

2457082

In [31]:
mask_packaging = df[['packaging', 'packaging_tags', 'packaging_en']].notnull().all(axis=1)
df.loc[mask_packaging, ['packaging', 'packaging_tags', 'packaging_en']]

,packaging,packaging_tags,packaging_en
32,"Plastique, Barquette","en:plastic,en:tray","Plastic,Tray"
73,Glass,en:glass,Glass
91,Boîte en métal,fr:boite-en-metal,fr:boite-en-metal
181,"Plastique, Bocal","en:plastic,en:jar","Plastic,Jar"
188,Plastique,en:plastic,Plastic
...,...,...,...
2830709,HDPE,"en:plastic,en:hdpe-high-density-polyethylene","Plastic,Hdpe-high-density-polyethylene"
2830731,"Kunststoff, Styropor","en:plastic,de:styropor","Plastic,de:styropor"
2830744,Boîte,en:box,Box
2830746,"Plastique, Sachet","en:plastic,en:bag","Plastic,Bag"


### brands

In [35]:
df['brands'].isna().sum()

1403611

In [33]:
df['brands_tags'].isna().sum()

1403739

In [36]:
mask_brands = df[['brands', 'brands_tags']].notnull().all(axis=1)
df.loc[mask_brands, ['brands', 'brands_tags']]

,brands,brands_tags
0,endives,endives
3,Danone,danone
11,courte paille,courte-paille
27,Japet Dels Bous,japet-dels-bous
29,Jeff de Bruges,jeff-de-bruges
...,...,...
2830750,Thiriet,thiriet
2830751,Harrys,harrys
2830752,Harrys,harrys
2830753,Harrys,harrys


### categories

In [38]:
df['categories'].isna().sum()

1619313

In [39]:
df['categories_tags'].isna().sum()

1619317

In [41]:
df['categories_en'].isna().sum()

1619318

In [42]:
mask_categories = df[['categories', 'categories_tags', 'categories_en']].notnull().all(axis=1)
df.loc[mask_categories, ['categories', 'categories_tags', 'categories_en']]

,categories,categories_tags,categories_en
3,"Produits laitiers, Desserts, Produits fermenté...","en:dairies,en:fermented-foods,en:desserts,en:f...","Dairies,Fermented foods,Desserts,Fermented mil..."
11,"Condiments, Sauces, Moutardes, en:groceries","en:condiments,en:sauces,en:mustards,en:groceries","Condiments,Sauces,Mustards,Groceries"
20,"Aliments et boissons à base de végétaux, Boiss...","en:plant-based-foods-and-beverages,en:beverage...","Plant-based foods and beverages,Beverages,Plan..."
21,"Snacks, Snacks sucrés, Confiseries, Massepain,...","en:snacks,en:sweet-snacks,en:confectioneries,e...","Snacks,Sweet snacks,Confectioneries,Marzipan,A..."
23,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...","Plant-based foods and beverages,Plant-based fo..."
...,...,...,...
2830750,"Desserts,Frozen foods,Frozen desserts","en:desserts,en:frozen-foods,en:frozen-desserts","Desserts,Frozen foods,Frozen desserts"
2830751,en:brioches,"en:snacks,en:sweet-snacks,en:viennoiseries,en:...","Snacks,Sweet snacks,Viennoiseries,Brioches"
2830752,en:brioches,"en:snacks,en:sweet-snacks,en:viennoiseries,en:...","Snacks,Sweet snacks,Viennoiseries,Brioches"
2830753,en:almond-butters,"en:plant-based-foods-and-beverages,en:plant-ba...","Plant-based foods and beverages,Plant-based fo..."


### Origins

In [12]:
df['origins'].isna().sum()

2719525

In [13]:
df['origins_tags'].isna().sum()

2719653

In [14]:
df['origins_en'].isna().sum()

2719661

In [37]:
mask_origins = df[['origins', 'origins_tags', 'origins_en']].notnull().all(axis=1)
df.loc[mask_origins, ['origins', 'origins_tags', 'origins_en']]

,origins,origins_tags,origins_en
2,Spain,en:spain,Spain
362,France,en:france,France
365,Québec,fr:quebec,fr:quebec
381,Québec,fr:quebec,fr:quebec
404,France,en:france,France
...,...,...,...
2825230,Espagne,en:spain,Spain
2825330,Hongrie,en:hungary,Hungary
2825385,România,en:romania,Romania
2825394,france,en:france,France


### Manufacturing_places

In [19]:
df['manufacturing_places'].isna().sum()

2666220

In [20]:
df['manufacturing_places_tags'].isna().sum()

2666287

In [48]:
mask_manufacturing = df[['manufacturing_places', 'manufacturing_places_tags']].notnull().all(axis=1)
df.loc[mask_manufacturing, ['manufacturing_places', 'manufacturing_places_tags']]

,manufacturing_places,manufacturing_places_tags
73,bénivay-ollon,benivay-ollon
181,France,france
362,France,france
365,Brossard Québec,brossard-quebec
381,Brossard Québec,brossard-quebec
...,...,...
2825330,Hongrie,hongrie
2825385,y,y
2825437,Hongrie,hongrie
2830731,"Meppen,Deutschland","meppen,deutschland"


### Labels

In [27]:
df['labels'].isna().sum()

2118373

In [28]:
df['labels_tags'].isna().sum()

2118378

In [29]:
df['labels_en'].isna().sum()

2118401

In [44]:
mask_manufacturing = df[['labels', 'labels_tags','labels_en']].notnull().all(axis=1)
df.loc[mask_manufacturing, ['labels', 'labels_tags','labels_en']]

,labels,labels_tags,labels_en
11,Delois france,fr:delois-france,fr:delois-france
29,Fabriqué en France,en:made-in-france,Made in France
42,"Sans arômes artificiels, Sans colorants, Sans ...","en:no-artificial-flavors,en:no-preservatives,e...","No artificial flavors,No preservatives,No colo..."
57,"Bio, Bio européen, FR-BIO-01","en:organic,en:eu-organic,en:fr-bio-01","Organic,EU Organic,FR-BIO-01"
62,"Organic, EU Organic, fr:ab-agriculture-biologique","en:organic,en:eu-organic,fr:ab-agriculture-bio...","Organic,EU Organic,AB Agriculture Biologique"
...,...,...,...
2830743,"Nutriscore, Nutriscore Grade A, en:high-proteins","en:high-proteins,en:nutriscore,en:nutriscore-g...","High proteins,Nutriscore,Nutriscore Grade A"
2830744,"No preservatives, Nutriscore, Nutriscore Grade...","en:no-preservatives,fr:fruits-et-legumes-de-fr...","No preservatives,fr:Fruits et Légumes de Franc..."
2830746,"Sans huile de palme, Réduction d'eau consommée...","en:no-palm-oil,fr:reduction-d-eau-consommee-da...","No palm oil,fr:reduction-d-eau-consommee-dans-..."
2830750,fr:Entrepreneurs + Engagés,fr:entrepreneurs-engages,fr:Entrepreneurs + Engagés


### emb_codes

In [45]:
df['emb_codes'].isna().sum()

2683839

In [46]:
df['emb_codes_tags'].isna().sum()

2683889

In [50]:
mask_emb_codes = df[['emb_codes', 'emb_codes_tags']].notnull().all(axis=1)
df.loc[mask_emb_codes, ['emb_codes', 'emb_codes_tags']]

,emb_codes,emb_codes_tags
45,FR 72.264.002 EC,fr-72-264-002-ec
59,FR 56.017.001 EC,fr-56-017-001-ec
73,13089c,13089c
87,FR 60.341.011 EC,fr-60-341-011-ec
127,FR 49.119.001 EC,fr-49-119-001-ec
...,...,...
2825389,EMB 74289B,emb-74289b
2825394,FR 31.091.010 EC,fr-31-091-010-ec
2825437,L700412,l700412
2830671,EMB 78291B,emb-78291b


### cities

In [61]:
df['cities'].isna().sum()

2830755

In [65]:
df['cities_tags'].isna().sum()

2743454

In [75]:
mask_cities = df[['cities_tags']].notna().all(axis=1)
df.loc[mask_cities, ['cities_tags']]

,cities_tags
45,sable-sur-sarthe-sarthe-france
59,bignan-morbihan-france
87,lagny-le-sec-oise-france
127,daumeray-maine-et-loire-france
129,saint-andiol-bouches-du-rhone-france
...,...
2825337,wimille-pas-de-calais-france
2825389,vallieres-haute-savoie-france
2825394,bruguieres-haute-garonne-france
2830671,guerville-yvelines-france


### countries

In [69]:
df['countries'].isna().sum()

7544

In [72]:
df['countries_tags'].isna().sum()

7549

In [73]:
df['countries_en'].isna().sum()

7549

In [74]:
mask_countries = df[['countries', 'countries_tags','countries_en']].notnull().all(axis=1)
df.loc[mask_countries, ['countries', 'countries_tags','countries_en']]

,countries,countries_tags,countries_en
0,en:france,en:france,France
1,en:de,en:germany,Germany
2,Spain,en:spain,Spain
3,France,en:france,France
4,France,en:france,France
...,...,...,...
2830750,France,en:france,France
2830751,en:fr,en:france,France
2830752,France,en:france,France
2830753,en:FR,en:france,France


### traces

In [76]:
df['traces'].isna().sum()

2706928

In [77]:
df['traces_tags'].isna().sum()

2676938

In [79]:
df['traces_en'].isna().sum()

2678667

In [80]:
mask_traces = df[['traces', 'traces_tags','traces_en']].notnull().all(axis=1)
df.loc[mask_traces, ['traces', 'traces_tags','traces_en']]

,traces,traces_tags,traces_en
193,"en:eggs,en:milk","en:eggs,en:milk","Eggs,Milk"
312,"en:eggs,en:gluten,en:milk,en:nuts,en:soybeans,...","en:eggs,en:gluten,en:milk,en:nuts,en:soybeans,...","Eggs,Gluten,Milk,Nuts,Soybeans,Oatmeal"
365,fr:CONTIENT : OEUF. LAIT. ANCHOIS. SOYA. BLE. ...,fr:contient-oeuf-lait-anchois-soya-ble-seigle-...,fr:contient-oeuf-lait-anchois-soya-ble-seigle-...
415,"en:gluten,en:soybeans,en:sulphur-dioxide-and-s...","en:gluten,en:soybeans,en:sulphur-dioxide-and-s...","Gluten,Soybeans,Sulphur dioxide and sulphites"
437,"en:gluten,fr:Peut contenir: Noix variés Soya ...","en:gluten,fr:peut-contenir-noix-varies-soya-la...","Gluten,fr:peut-contenir-noix-varies-soya-lait-..."
...,...,...,...
2825330,fr:crevette céleri œuf lait sésame,fr:crevette-celeri-oeuf-lait-sesame,fr:crevette-celeri-oeuf-lait-sesame
2825437,"en:celery,en:crustaceans,en:molluscs,en:mustar...","en:celery,en:crustaceans,en:molluscs,en:mustar...","Celery,Crustaceans,Molluscs,Mustard,Sesame seeds"
2830746,"en:milk,en:mustard,en:nuts,en:sesame-seeds,en:...","en:milk,en:mustard,en:nuts,en:sesame-seeds,en:...","Milk,Mustard,Nuts,Sesame seeds,Soybeans"
2830749,en:celery,en:celery,Celery


### additives

In [81]:
df['additives'].isna().sum()

2830753

In [82]:
df['additives_tags'].isna().sum()

2358337

In [83]:
df['additives_en'].isna().sum()

2358337

In [84]:
mask_traces = df[['additives', 'additives_tags','additives_en']].notnull().all(axis=1)
df.loc[mask_traces, ['additives', 'additives_tags','additives_en']]

,additives,additives_tags,additives_en
863471,[ 59-garniture -> fr:59-garniture ] [ 59 ->...,"en:e250,en:e270,en:e301,en:e579","E250 - Sodium nitrite,E270 - Lactic acid,E301 ..."


### Suppression colonnes _tags et etc

In [92]:
cols_to_drop = ['packaging', 'packaging_tags', 'brands_tags', 'categories', 'categories_tags',
                'origins', 'origins_tags', 'manufacturing_places_tags', 'labels', 'labels_tags',
                'emb_codes_tags', 'cities', 'countries', 'countries_tags', 'traces', 'traces_tags',
                'additives', 'additives_tags'
                ]

In [93]:
df_withoutduplicatefeatures = df.drop(columns=cols_to_drop)
df_withoutduplicatefeatures

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,product_name,abbreviated_product_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,nutrinet-sante,jeunes pousses,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1656948610,2022-07-04T15:30:10Z,1656948613,2022-07-04T15:30:13Z,kiliweb,Andrè,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1656488106,2022-06-29T07:35:06Z,stephane,L.casei,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1654250311,2022-06-03T09:58:31Z,1654270474,2022-06-03T15:34:34Z,quentinbrd,Skyr,NaN,...,NaN,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,kiliweb,Vitória crackers,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830750,3292590606497,http://world-en.openfoodfacts.org/product/3292...,kiliweb,1509909912,2017-11-05T19:25:12Z,1681141329,2023-04-10T15:42:09Z,moon-rabbit,Instant feerique,NaN,...,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830751,35033264,http://world-en.openfoodfacts.org/product/3503...,kiliweb,1583067829,2020-03-01T13:03:49Z,1681320470,2023-04-12T17:27:50Z,xdesnoyers,DooWap,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830752,39240958,http://world-en.openfoodfacts.org/product/3924...,kiliweb,1522995251,2018-04-06T06:14:11Z,1681320420,2023-04-12T17:27:00Z,xdesnoyers,Doowap,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830753,72355671,http://world-en.openfoodfacts.org/product/7235...,kiliweb,1558252418,2019-05-19T07:53:38Z,1680877496,2023-04-07T14:24:56Z,itsjustruby,Almond Butter,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
df_withoutduplicatefeatures.to_csv("df_withoutduplicatefeatures.csv", sep='\t', encoding='utf-8', index=False)

In [95]:
df_withoutduplicatefeatures.dtypes

code                   object
url                    object
creator                object
created_t               int64
created_datetime       object
                       ...   
choline_100g          float64
phylloquinone_100g    float64
beta-glucan_100g      float64
inositol_100g         float64
carnitine_100g        float64
Length: 183, dtype: object

Autres colonnes à suppr

In [125]:
df['created_t'].isna().sum()

0

In [96]:
df['created_datetime'].isna().sum()

0

In [97]:
df['last_modified_t'].isna().sum()

0

In [98]:
df['last_modified_datetime'].isna().sum()

0

In [99]:
df['main_category'].isna().sum()

1619317

In [107]:
df['main_category_en'].isna().sum()

1619318

In [101]:
df['image_url'].isna().sum()

531206

In [102]:
df['image_small_url'].isna().sum()

531206

In [114]:
pd.options.display.max_colwidth = 100

In [116]:
mask_traces = df[['code','image_url','image_small_url']].notnull().all(axis=1)
df.loc[mask_traces, ['code','image_url','image_small_url']].head(20)

,code,image_url,image_small_url
2,00000000000003429145,https://images.openfoodfacts.org/images/products/000/000/000/00003429145/front_es.3.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/00003429145/front_es.3.200.jpg
3,00000000000026772226,https://images.openfoodfacts.org/images/products/000/000/000/00026772226/front_fr.3.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/00026772226/front_fr.3.200.jpg
4,0000000000017,https://images.openfoodfacts.org/images/products/000/000/000/0017/front_fr.4.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/0017/front_fr.4.200.jpg
5,0000000000031,https://images.openfoodfacts.org/images/products/000/000/000/0031/front_fr.3.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/0031/front_fr.3.200.jpg
6,000000000003327986,https://images.openfoodfacts.org/images/products/000/000/000/003327986/front_es.3.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/003327986/front_es.3.200.jpg
7,000000000004128579,https://images.openfoodfacts.org/images/products/000/000/000/004128579/front_es.3.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/004128579/front_es.3.200.jpg
8,000000000004622327,https://images.openfoodfacts.org/images/products/000/000/000/004622327/front_es.3.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/004622327/front_es.3.200.jpg
9,000000000006021,https://images.openfoodfacts.org/images/products/000/000/000/006021/front_en.3.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/006021/front_en.3.200.jpg
11,0000000000100,https://images.openfoodfacts.org/images/products/000/000/000/0100/front.3.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/0100/front.3.200.jpg
12,0000000000123,https://images.openfoodfacts.org/images/products/000/000/000/0123/front_fr.4.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/0123/front_fr.4.200.jpg


In [121]:
cols_to_drop2 = ['created_datetime', 'last_modified_t', 'last_modified_datetime', 'main_category', 'image_url'
                ]

In [122]:
df_withoutduplicatefeatures2 = df_withoutduplicatefeatures.drop(columns=cols_to_drop2)
df_withoutduplicatefeatures2

,code,url,creator,created_t,last_modified_by,product_name,abbreviated_product_name,generic_name,quantity,packaging_en,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/00000000000000225/jeunes-pousses-endives,nutrinet-sante,1623855208,nutrinet-sante,jeunes pousses,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000000000000207025004/andre,kiliweb,1656948610,kiliweb,Andrè,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,http://world-en.openfoodfacts.org/product/00000000000003429145/l-casei,kiliweb,1630483911,stephane,L.casei,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000026772226,http://world-en.openfoodfacts.org/product/00000000000026772226/skyr-danone,kiliweb,1654250311,quentinbrd,Skyr,NaN,NaN,480 g,NaN,...,NaN,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000017,http://world-en.openfoodfacts.org/product/0000000000017/vitoria-crackers,kiliweb,1529059080,kiliweb,Vitória crackers,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830750,3292590606497,http://world-en.openfoodfacts.org/product/3292590606497/instant-feerique-thiriet,kiliweb,1509909912,moon-rabbit,Instant feerique,NaN,NaN,NaN,NaN,...,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830751,35033264,http://world-en.openfoodfacts.org/product/35033264/doowap-harrys,kiliweb,1583067829,xdesnoyers,DooWap,NaN,NaN,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830752,39240958,http://world-en.openfoodfacts.org/product/39240958/doowap-harrys,kiliweb,1522995251,xdesnoyers,Doowap,NaN,NaN,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830753,72355671,http://world-en.openfoodfacts.org/product/72355671/almond-butter-harrys,kiliweb,1558252418,itsjustruby,Almond Butter,NaN,NaN,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
df_withoutduplicatefeatures2.to_csv("df_withoutduplicatefeatures2.csv", sep='\t', encoding='utf-8', index=False)

In [124]:
df_withoutduplicatefeatures2

,code,url,creator,created_t,last_modified_by,product_name,abbreviated_product_name,generic_name,quantity,packaging_en,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/00000000000000225/jeunes-pousses-endives,nutrinet-sante,1623855208,nutrinet-sante,jeunes pousses,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000000000000207025004/andre,kiliweb,1656948610,kiliweb,Andrè,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,http://world-en.openfoodfacts.org/product/00000000000003429145/l-casei,kiliweb,1630483911,stephane,L.casei,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000026772226,http://world-en.openfoodfacts.org/product/00000000000026772226/skyr-danone,kiliweb,1654250311,quentinbrd,Skyr,NaN,NaN,480 g,NaN,...,NaN,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000017,http://world-en.openfoodfacts.org/product/0000000000017/vitoria-crackers,kiliweb,1529059080,kiliweb,Vitória crackers,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830750,3292590606497,http://world-en.openfoodfacts.org/product/3292590606497/instant-feerique-thiriet,kiliweb,1509909912,moon-rabbit,Instant feerique,NaN,NaN,NaN,NaN,...,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830751,35033264,http://world-en.openfoodfacts.org/product/35033264/doowap-harrys,kiliweb,1583067829,xdesnoyers,DooWap,NaN,NaN,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830752,39240958,http://world-en.openfoodfacts.org/product/39240958/doowap-harrys,kiliweb,1522995251,xdesnoyers,Doowap,NaN,NaN,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830753,72355671,http://world-en.openfoodfacts.org/product/72355671/almond-butter-harrys,kiliweb,1558252418,itsjustruby,Almond Butter,NaN,NaN,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
uu = df_withoutduplicatefeatures2.select_dtypes('number')
uu.dtypes

created_t               int64
allergens_en          float64
serving_quantity      float64
additives_n           float64
nutriscore_score      float64
                       ...   
choline_100g          float64
phylloquinone_100g    float64
beta-glucan_100g      float64
inositol_100g         float64
carnitine_100g        float64
Length: 127, dtype: object

In [127]:
df['created_t'].str.len().max()

AttributeError: Can only use .str accessor with string values!

In [128]:
df['created_t'].max()

1681437015

## df_nutrition

In [8]:
df_nutrition = df.filter(regex='^(code|product_name|.*_100g)$')
df_nutrition

,code,product_name,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,jeunes pousses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,Andrè,NaN,165.000000,690.0,NaN,2.00000,2.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,L.casei,NaN,NaN,NaN,NaN,1.40000,0.900000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000026772226,Skyr,NaN,57.000000,238.0,NaN,0.20000,0.100000,NaN,NaN,...,NaN,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000017,Vitória crackers,NaN,375.000000,1569.0,NaN,7.00000,3.080000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830750,3292590606497,Instant feerique,1019.0,243.000000,1019.0,NaN,10.00000,5.900000,NaN,NaN,...,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830751,35033264,DooWap,NaN,340.000000,1423.0,NaN,12.10000,3.900000,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830752,39240958,Doowap,NaN,335.000000,1402.0,NaN,11.50000,3.300000,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830753,72355671,Almond Butter,NaN,612.903226,2564.0,NaN,54.83871,6.451613,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_nutrition.to_csv("df_nutrition.csv", sep='\t', encoding='utf-8', index=False)

In [10]:
%%time

df_nutrition=pd.read_csv("df_nutrition.csv",
              sep="\t", encoding="utf-8", low_memory=False)

CPU times: total: 20.8 s
Wall time: 21.1 s


In [11]:
df_nutrition

,code,product_name,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,jeunes pousses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,Andrè,NaN,165.000000,690.0,NaN,2.00000,2.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000000003429145,L.casei,NaN,NaN,NaN,NaN,1.40000,0.900000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000026772226,Skyr,NaN,57.000000,238.0,NaN,0.20000,0.100000,NaN,NaN,...,NaN,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000017,Vitória crackers,NaN,375.000000,1569.0,NaN,7.00000,3.080000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830750,3292590606497,Instant feerique,1019.0,243.000000,1019.0,NaN,10.00000,5.900000,NaN,NaN,...,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830751,35033264,DooWap,NaN,340.000000,1423.0,NaN,12.10000,3.900000,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830752,39240958,Doowap,NaN,335.000000,1402.0,NaN,11.50000,3.300000,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2830753,72355671,Almond Butter,NaN,612.903226,2564.0,NaN,54.83871,6.451613,NaN,NaN,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 0.Filtrer le dataframe

In [ ]:
from t0_datafilter import datas_filter

In [ ]:
filter = datas_filter()

filter.downcast(df, show_saved_memory=True)

# filter exemples
print("Mise en place des filtres :")
print("Colones numériques : ")
print(list(filter.filter(df, type="number", nan_percent=34).columns))
print("Colones ordinales : ")
#print(list(filter.filter(df, type="ordinal", ordinal_features_names=["i"]).columns))
print("Colones non-ordinales (max 2): ")
print(list(filter.filter(df, type="non-ordinal", category_count=2).columns))
print("Colones catégorielles : ")
print(list(filter.filter(df, type="categorical").columns))

## 1.0.Nettoyer le jeu de données des valeurs problématiques

In [ ]:
def count_duplicated_values(df):
        """Count duplicated values for each feature
        Returns:
            df (DataFrame): return sum of duplicated values
        @Author: Thomas PAYAN
        """
        return df.duplicated().sum()

In [ ]:
def count_duplicate_rows(df):
    """Compte le nombre de lignes en doublon dans un DataFrame"""
    duplicates = df.duplicated()
    return duplicates.sum()

count_duplicate_rows(df)

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing.sort_values(ascending=False)

threshold_view = 2

ax = sns.barplot(x = percent_missing[percent_missing.values > threshold_view], 
                 y=percent_missing[percent_missing.values > threshold_view].index, 
                 orient='h');
ax.set_title(f"Répartition du pourcentage de valeurs manquantes supérieures au seuil de 2%")

In [ ]:
def drop_duplicated_values(df):
        """Drop duplicated values
        Returns:
            df (DataFrame): return dataframe without duplicated values
        @Author: Thomas PAYAN
        """
        print("\nRows number before drop duplicated values : %s" % len(df))
        if(df.count_duplicated_values() > 0):
            df.drop_duplicates(inplace=True)
        print("\nRows number after drop duplicated values : %s" % len(df))
        return df

In [ ]:
from t1_0_cleandata import drop_columns_with_missing_values, impute_missing_values

In [ ]:
df = drop_columns_with_missing_values(df, threshold=0.95)

In [ ]:
df.dtypes

In [ ]:
df.select_dtypes(include=np.number).columns.tolist()

In [ ]:
columns = ['created_t',
 'last_modified_t',
 'serving_quantity',
 'additives_n',
 'nutriscore_score',
 'nova_group',
 'ecoscore_score',
 'product_quantity',
 'unique_scans_n',
 'completeness',
 'last_image_t',
 'energy-kcal_100g',
 'energy_100g',
 'fat_100g',
 'saturated-fat_100g',
 'trans-fat_100g',
 'cholesterol_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fiber_100g',
 'proteins_100g',
 'salt_100g',
 'sodium_100g',
 'vitamin-a_100g',
 'vitamin-c_100g',
 'calcium_100g',
 'iron_100g',
 'fruits-vegetables-nuts-estimate-from-ingredients_100g',
 'nutrition-score-fr_100g']

In [ ]:
impute_missing_values(df, columns, missing_values='NaN', n_neighbors=5, weights='uniform')